In [1]:
!pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 39.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.0 MB/s eta 0:00:00:00:01


In [2]:
import pennylane as qml
from pennylane.operation import Operation
import numpy as np
import tensorflow as tf

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train[:25000]
y_train = y_train[:25000]
x_test = x_test[-5000:]
y_test = y_test[-5000:]

# Rescale the images
x_train = x_train.reshape(-1, 784) / 255.0
x_test = x_test.reshape(-1, 784) / 255.0

def filter_36(x, y):
    keep = (y == 3) | (y == 6)
    x, y = x[keep], y[keep]
    y = y == 3
    return x, y

x_train, y_train = filter_36(x_train, y_train)
x_test, y_test = filter_36(x_test, y_test)

class RBSGate(Operation):
    num_params = 1
    num_wires = 2
    par_domain = 'R'

    def __init__(self, theta, wires):
        super().__init__(theta, wires=wires)
        self.theta = theta

    @staticmethod
    def compute_matrix(theta):
        cos = tf.cos(theta)
        sin = tf.sin(theta)
        return tf.convert_to_tensor([
            [1, 0, 0, 0],
            [0, cos, sin, 0],
            [0, -sin, cos, 0],
            [0, 0, 0, 1]
        ], dtype=tf.float64)

    def adjoint(self):
        return RBSGate(-self.parameters[0], wires=self.wires)

    def label(self, decimals=None, base_label=None, **kwargs):
        theta = self.parameters[0]
        return f"RBS({theta:.2f})"

def convert_array(X):
    alphas = tf.zeros(X.shape[:-1] + (X.shape[-1]-1,), dtype=X.dtype)
    X_normd = tf.linalg.l2_normalize(X, axis=-1)
    for i in range(X.shape[-1]-1):
        prod_sin_alphas = tf.reduce_prod(tf.sin(alphas[..., :i]), axis=-1)
        updated_value = tf.acos(X_normd[..., i] / prod_sin_alphas)
        indices = tf.constant([[i]])
        updates = tf.reshape(updated_value, [1])
        alphas = tf.tensor_scatter_nd_update(alphas, indices, updates)
    return alphas

def vector_loader(alphas, wires=None, is_x=True, is_conjugate=False):
    if wires is None:
        wires = list(range(len(alphas) + 1))
    if is_x and not is_conjugate:
        qml.PauliX(wires=wires[0])
    if is_conjugate:
        for i in range(len(wires) - 2, -1, -1):
            qml.apply(RBSGate(-alphas[i], wires=[wires[i], wires[i+1]]))
    else:
        for i in range(len(wires) - 1):
            qml.apply(RBSGate(alphas[i], wires=[wires[i], wires[i+1]]))
    if is_x and is_conjugate:
        qml.PauliX(wires=wires[0])

def pyramid_circuit(parameters, wires=None):
    # If wires is None, use all qubits in the circuit
    if wires is None:
        length = len(qml.device.wires)
    else:
        # If wires is not None, ensure it's a list of qubits
        length = len(wires)

    k = 0

    for i in range(2 * length - 2):
        j = length - abs(length - 1 - i)

        if i % 2:
            for _ in range(j):
                if _ % 2 == 0 and k < len(parameters):
                    qml.apply(RBSGate(parameters[k], wires=([wires[_], wires[_ + 1]])))
                    k += 1
        else:
            for _ in range(j):
                if _ % 2 and k < len(parameters):
                    qml.apply(RBSGate(parameters[k], wires=([wires[_], wires[_ + 1]])))
                    k += 1

2024-08-20 15:06:51.839798: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 15:06:51.839925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 15:06:51.987971: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
dev1 = qml.device('default.qubit', wires=4)

@qml.qnode(dev1, interface='tf', diff_method='backprop')
def quantum_model_pyramid(inputs, weights):
    inputs = tf.cast(inputs, tf.float64)
    weights = tf.cast(weights, tf.float64)
    vector_loader(convert_array(inputs), wires=range(4))
    pyramid_circuit(weights, wires=range(4))
    return [qml.expval(qml.PauliZ(wire)) for wire in range(4)]

weights_init_pyramid = np.random.normal(size=(6,), scale=np.pi/4)
weights_pyramid = tf.Variable(weights_init_pyramid, dtype=tf.float64)
class HybridModel(tf.keras.Model):
    def __init__(self, quantum_model_pyramid):
        super(HybridModel, self).__init__()
        self.quantum_model_pyramid = quantum_model_pyramid
        # Classical layer to project 7x7 patches into a 4D space
        self.classical_nn_patch = tf.keras.layers.Dense(4, activation='relu', dtype=tf.float64)
        # Additional classical layers after the pyramid circuit
        self.classical_nn_1 = tf.keras.layers.Dense(8, activation='relu', dtype=tf.float64)
        # Change the final output layer to use sigmoid activation
        self.classical_nn_2 = tf.keras.layers.Dense(1, activation='sigmoid', dtype=tf.float64)

    def build(self, input_shape):
        self.quantum_weights_p = self.add_weight(shape=(6,), initializer='random_normal', trainable=True, dtype=tf.float64)

    def call(self, inputs):
        inputs = tf.cast(inputs, tf.float64)
        # Reshape the flattened input (28x28 = 784) back to its 2D form (28x28)
        inputs = tf.reshape(inputs, [-1, 28, 28, 1])

        # Extract 7x7 patches from the 28x28 input
        patches = tf.image.extract_patches(images=inputs,
                                           sizes=[1, 7, 7, 1],
                                           strides=[1, 7, 7, 1],
                                           rates=[1, 1, 1, 1],
                                           padding='VALID')
        # Flatten each 7x7 patch into a 49-dimensional vector
        patches = tf.reshape(patches, [-1, 49])

        # Apply the classical NN to project each patch into a 4D space
        projected_patches = self.classical_nn_patch(patches)
        
        # Reshape the projected patches for processing by the pyramid circuit
        projected_patches = tf.reshape(projected_patches, [-1, 4])

        # Apply the pyramid circuit to each 4D space
        quantum_outputs = tf.map_fn(lambda x: tf.stack(self.quantum_model_pyramid(x, self.quantum_weights_p)),
                                    projected_patches, dtype=tf.float64)

        quantum_outputs = tf.where(tf.math.is_nan(quantum_outputs), tf.zeros_like(quantum_outputs), quantum_outputs)
        
        # Flatten the quantum outputs
        quantum_outputs = tf.reshape(quantum_outputs, [-1, 64])

        # Additional classical NN layers after pyramid outputs
        nn_output = self.classical_nn_1(quantum_outputs)
        nn_output = self.classical_nn_2(nn_output)
        
        return nn_output


# Create an instance of the modified HybridModel
model = HybridModel(quantum_model_pyramid)


# Use binary cross-entropy loss
loss_fn = tf.keras.losses.BinaryCrossentropy()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

x_train_small_tensor = tf.convert_to_tensor(x_train, dtype=tf.float64)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float64)  # Labels should be in range [0, 1] for binary cross-entropy

@tf.function
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_fn(targets, predictions)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(predictions), targets), tf.float64))
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, accuracy

@tf.function
def validation_step(inputs, targets):
    predictions = model(inputs)
    loss = loss_fn(targets, predictions)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(predictions), targets), tf.float64))
    return loss, accuracy

x_test_small_tensor = tf.convert_to_tensor(x_test, dtype=tf.float64)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float64)

epochs = 30
for epoch in range(epochs):
    loss, acc = train_step(x_train_small_tensor, y_train_tensor)
    val_loss, val_acc = validation_step(x_test_small_tensor, y_test_tensor)
    print(f'Epoch {epoch + 1} ----------------------------------- \n Train Loss: {loss.numpy()}, Train Accuracy: {acc.numpy()},  Validation Loss: {val_loss.numpy()}, Validation Accuracy: {val_acc.numpy()}')


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: cannot use assignment expressions with function call (__autograph_generated_filern7gj5h4.py, line 96)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function _gcd_import at 0x7b415fc37400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experim